In [1]:
# importing csv module
import csv
  
# csv file name
filename = "TestData.csv"
  
# initializing the titles and rows list
fields = []
rows = []
  
# reading csv file
with open(filename, 'r') as csvfile:
    # creating a csv reader object
    csvreader = csv.reader(csvfile)
      
    # extracting field names through first row
    fields = next(csvreader)
  
    # extracting each data row one by one
    for row in csvreader:
        rows.append(row)

In [2]:
#Extracting CSV Headers
NewFields = [i.split('\t') for i in fields][0]
NewFields

['Indel',
 'Chr-Pos',
 'MultiSampleCount',
 'MultiAlleleCount',
 'NearbyIndelCount25bp',
 'RecurrentIndel']

In [3]:
#Creating Rows for pandas DataFrame
NewRows = []
for row in rows:
    for i in row:
        NewRows.append(i.split('\t'))

In [4]:
#Load CSV data in as DataFrame
import pandas as pd
df = pd.DataFrame(NewRows,columns = NewFields)
df

,Indel,Chr-Pos,MultiSampleCount,MultiAlleleCount,NearbyIndelCount25bp,RecurrentIndel
0,1-100000989-A-ATC,1-100000989,11,1,1,TRUE
1,1-100001151-GT-G,1-100001151,1,1,2,TRUE
2,1-100001168-TG-T,1-100001168,1,1,2,TRUE
3,1-100001497-TA-T,1-100001497,1,1,1,FALSE
4,1-100001671-C-CTT,1-100001671,5,4,1,TRUE
...,...,...,...,...,...,...
15022039,Y-9992125-AT-A,Y-9992125,3,1,1,TRUE
15022040,Y-9992151-TA-T,Y-9992151,6,1,2,TRUE
15022041,Y-9992174-AC-A,Y-9992174,7,1,2,TRUE
15022042,Y-9994833-TAATG-T,Y-9994833,2,1,1,TRUE


In [5]:
Chrom_Dict = {}
for i in range(0,len(df['Chr-Pos'].tolist())):
    Chromosome,Pos = df['Chr-Pos'][i].split('-')
    if Chromosome not in Chrom_Dict:
        Chrom_Dict[Chromosome] = [int(Pos)]
    else:
        Chrom_Dict[Chromosome].append(int(Pos))

In [6]:
for key, value in Chrom_Dict.items():
    vals = sorted(value)
    Chrom_Dict[key] = vals

In [7]:
ChromPosDifference = {}
import numpy as np
for key,value in Chrom_Dict.items():
    val =  abs(np.diff(value))
    ChromPosDifference[key] = val.tolist()

In [8]:
CounterList = []
for key, value in ChromPosDifference.items():
    counter_dict = {}
    for i in value:
        if i in counter_dict:
            counter_dict[i] += 1
        else:
            counter_dict[i] = 1
    CounterList.append(counter_dict)

In [9]:
Chromosomes = list(Chrom_Dict.keys())
PositionCounter = dict(zip(Chromosomes, CounterList))

In [10]:
for key, value in PositionCounter.items():
    newval = dict(sorted(PositionCounter[key].items(), key=lambda item: item[0]))
    PositionCounter[key] = newval

In [11]:
X = PositionCounter['X']
Y = PositionCounter['Y']

In [12]:
del(PositionCounter['X'])
del(PositionCounter['Y'])

In [13]:
PosCounts = {}
for key, value in sorted(PositionCounter.items()):
    keys = int(key)
    PosCounts[keys] = value

In [49]:
PosCountSorted = dict(sorted(PosCounts.items()))

In [50]:
PosCountSorted['X'] = X
PosCountSorted['Y'] = Y

In [440]:
TupleList = []
for key, val in PosCountSorted.items():
    ChromDict = {}
    ChromDict[key] = val
    for CDkeys, CDvals in ChromDict.items():
        CDsubkeys = list(CDvals.keys())
        CDsubvals = list(CDvals.values())
        for i in range(0,len(CDsubkeys)):
            TupleList.append((CDsubkeys[i],CDsubvals[i]))

In [500]:
SplitVals = []
plitpoint = []
for k,g in itertools.groupby(TupleList,lambda x:x[0] in (0,)): 
    if k:
        splitpoint = list(g)
    else:
        SplitVals.append(splitpoint + list(g))

In [592]:
DictList = []
for i in SplitVals:
    BPs = []
    Freqs = []
    for tup in i:
        BPs.append(tup[0])
        Freqs.append(tup[1])
        
    BP_range = []  
    BPCumFreq = []
    
    for l in range(0,len(BPs)):
        if l%10 == 0:
            BP_rows = (BPs[l:l+10])
            if len(BP_rows) == 10:
                BP_rows = BP_rows
                BP_rows = (BP_rows[0],BP_rows[-1])
                BP_range.append(BP_rows)
    for j in range(0,len(Freqs)):
        if j%10 == 0:
            FreqRows = (Freqs[j:j+10])
            if len(FreqRows) == 10:
                FreqRows = FreqRows
                sums = sum(FreqRows)
                BPCumFreq.append(sums)
    Dicts = dict(zip(BP_range,BPCumFreq))
    DictList.append(Dicts)
    
                
        

In [593]:
Chromosomes = list(PosCountSorted.keys())

In [594]:
Range_Freq_Dist = dict(zip(Chromosomes,DictList))

In [598]:
for key, value in Range_Freq_Dist.items():
    sums = value.values()
    sums = sum(sums)
    for keys, vals in value.items():
        value[keys] = round((vals/sums)*100,3)

In [599]:
Range_Freq_Dist

{1: {(0, 9): 38.175,
  (10, 19): 6.68,
  (20, 29): 4.396,
  (30, 39): 3.449,
  (40, 49): 2.868,
  (50, 59): 2.494,
  (60, 69): 2.233,
  (70, 79): 2.0,
  (80, 89): 1.818,
  (90, 99): 1.65,
  (100, 109): 1.512,
  (110, 119): 1.394,
  (120, 129): 1.271,
  (130, 139): 1.215,
  (140, 149): 1.127,
  (150, 159): 1.114,
  (160, 169): 1.092,
  (170, 179): 0.996,
  (180, 189): 0.879,
  (190, 199): 0.809,
  (200, 209): 0.755,
  (210, 219): 0.729,
  (220, 229): 0.7,
  (230, 239): 0.669,
  (240, 249): 0.641,
  (250, 259): 0.617,
  (260, 269): 0.569,
  (270, 279): 0.534,
  (280, 289): 0.526,
  (290, 299): 0.523,
  (300, 309): 0.507,
  (310, 319): 0.471,
  (320, 329): 0.439,
  (330, 339): 0.422,
  (340, 349): 0.403,
  (350, 359): 0.379,
  (360, 369): 0.375,
  (370, 379): 0.357,
  (380, 389): 0.338,
  (390, 399): 0.323,
  (400, 409): 0.31,
  (410, 419): 0.306,
  (420, 429): 0.29,
  (430, 439): 0.271,
  (440, 449): 0.275,
  (450, 459): 0.264,
  (460, 469): 0.263,
  (470, 479): 0.247,
  (480, 489): 0.24